In [1]:
using BenchmarkTools
using Distributions

function pvalue_one_sided_naive(k, n, p=1/2)
    pval = 0.0
    for i in k:n
        pval += binomial(n, i) * p^i * (1-p)^(n-i)
    end
    pval
end

pvalue_one_sided_ccdf(k, n, p=1/2) = ccdf(Binomial(n, p), k-1)

@show pvalue_one_sided_naive(21, 30, 0.5)
@show pvalue_one_sided_ccdf(21, 30, 0.5);

pvalue_one_sided_naive(21, 30, 0.5) = 0.02138697262853384
pvalue_one_sided_ccdf(21, 30, 0.5) = 0.02138697262853383


In [2]:
@btime pvalue_one_sided_naive(21, 30, 0.5)
@btime pvalue_one_sided_ccdf(21, 30, 0.5);

  563.784 ns (0 allocations: 0 bytes)
  152.848 ns (0 allocations: 0 bytes)


In [3]:
@btime pvalue_one_sided_naive(16, 30, 0.5)
@btime pvalue_one_sided_ccdf(16, 30, 0.5);

  1.290 μs (0 allocations: 0 bytes)
  159.949 ns (0 allocations: 0 bytes)


In [4]:
pvalue_one_sided_naive(60, 100, 0.5)

LoadError: OverflowError: binomial(100, 60) overflows

In [5]:
using SpecialFunctions

function pvalue_one_sided_sf1(k, n, p=1/2)
    pval = 0.0
    for i in k:n
        pval += exp(logabsbinomial(n, i)[1]) * p^i * (1-p)^(n-i)
    end
    pval
end

@show pvalue_one_sided_sf1(60, 100, 0.5)
@show pvalue_one_sided_ccdf(60, 100, 0.5);

pvalue_one_sided_sf1(60, 100, 0.5) = 0.02844396682049029
pvalue_one_sided_ccdf(60, 100, 0.5) = 0.02844396682049039


In [6]:
@btime pvalue_one_sided_sf1(60, 100, 0.5)
@btime pvalue_one_sided_ccdf(60, 100, 0.5);

  3.638 μs (0 allocations: 0 bytes)
  161.269 ns (0 allocations: 0 bytes)


In [7]:
pvalue_one_sided_sf1(501000, 10^6, 0.5)

NaN

In [8]:
using SpecialFunctions

function pvalue_one_sided_sf2(k, n, p=1/2)
    pval = 0.0
    for i in k:n
        pval += exp(logabsbinomial(n, i)[1] + i*log(p) + (n-i)*log(1-p))
    end
    pval
end

@show pvalue_one_sided_sf1(60, 100, 0.5)
@show pvalue_one_sided_sf2(60, 100, 0.5)
@show pvalue_one_sided_ccdf(60, 100, 0.5);

pvalue_one_sided_sf1(60, 100, 0.5) = 0.02844396682049029
pvalue_one_sided_sf2(60, 100, 0.5) = 0.02844396682049034
pvalue_one_sided_ccdf(60, 100, 0.5) = 0.02844396682049039


In [9]:
@btime pvalue_one_sided_sf1(60, 100, 0.5)
@btime pvalue_one_sided_sf2(60, 100, 0.5)
@btime pvalue_one_sided_ccdf(60, 100, 0.5);

  3.562 μs (0 allocations: 0 bytes)
  3.475 μs (0 allocations: 0 bytes)
  161.010 ns (0 allocations: 0 bytes)


In [10]:
@show pvalue_one_sided_sf2(501000, 10^6, 0.5)
@show pvalue_one_sided_ccdf(501000, 10^6, 0.5);

pvalue_one_sided_sf2(501000, 10 ^ 6, 0.5) = 0.022804149931594936
pvalue_one_sided_ccdf(501000, 10 ^ 6, 0.5) = 0.02280414993269104


In [11]:
@btime pvalue_one_sided_sf2(501000, 10^6, 0.5)
@btime pvalue_one_sided_ccdf(501000, 10^6, 0.5);

  39.481 ms (0 allocations: 0 bytes)
  406.533 ns (4 allocations: 960 bytes)


In [12]:
ccdf(Binomial(30, 0.5), 20)

0.02138697262853383

In [13]:
@which ccdf(Binomial(30, 0.5), 20)

ccdf(d::Binomial, x::Real)
     @ Distributions D:\.julia\packages\Distributions\UaWBm\src\univariates.jl:649

In [14]:
Distributions.binomccdf(30, 0.5, 20)

0.02138697262853383

In [15]:
@which Distributions.binomccdf(30, 0.5, 20)

binomccdf(n::Real, p::Real, k::Real)
     @ StatsFuns D:\.julia\packages\StatsFuns\atA5S\src\distrs\binom.jl:29

In [16]:
Distributions.betacdf(20+1, 30-20, 0.5)

0.02138697262853383

In [17]:
@which Distributions.betacdf(20+1, 30-20, 0.5)

betacdf(α::Real, β::Real, x::Real)
     @ StatsFuns D:\.julia\packages\StatsFuns\atA5S\src\distrs\beta.jl:29

In [18]:
beta_inc(20+1, 30-20, 0.5)

(0.02138697262853383, 0.9786130273714662)

In [19]:
@which beta_inc(20+1, 30-20, 0.5)

beta_inc(a::Real, b::Real, x::Real)
     @ SpecialFunctions D:\.julia\packages\SpecialFunctions\QH8rV\src\beta_inc.jl:731

In [20]:
SpecialFunctions._beta_inc(float(20+1), float(30-20), 0.5)

(0.02138697262853383, 0.9786130273714662)

In [21]:
@which SpecialFunctions._beta_inc(float(20+1), float(30-20), 0.5)

_beta_inc(a::Float64, b::Float64, x::Float64)
     @ SpecialFunctions D:\.julia\packages\SpecialFunctions\QH8rV\src\beta_inc.jl:738

https://github.com/JuliaMath/SpecialFunctions.jl/blob/master/src/beta_inc.jl#L738